# Example usage of the `torchTextClassifiers` library

*Warning*

*`torchTextClassifiers` library is still under active development. Have a
regular look to <https://github.com/inseefrlab/torchTextClassifiers> for
latest information.*

To install package, you can run the following snippet

In [6]:
# Stable version
%uv pip install --system .. 
%uv pip install --system captum unidecode nltk


Using Python 3.13.5 environment at: /opt/python
Resolved 42 packages in 70ms                                         
   Building torchtextclassifiers @ file:///home/onyxia/work/torchTextClassifiers
      Built torchtextclassifiers @ file:///home/onyxia/work/torchTextClassifiers
Prepared 1 package in 28ms                                               
Uninstalled 1 package in 1ms
░░░░░░░░░░░░░░░░░░░░ [0/1] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 3mss==0.0.0.dev0 (from file:///home/o
 ~ torchtextclassifiers==0.0.0.dev0 (from file:///home/onyxia/work/torchTextClassifiers)
Note: you may need to restart the kernel to use updated packa

# Load and preprocess data

In that guide, we propose to illustrate main package functionalities
using that `DataFrame`:

In [1]:
import pandas as pd
df = pd.read_parquet("https://minio.lab.sspcloud.fr/projet-ape/extractions/20241027_sirene4.parquet")
df = df.sample(10000)

Our goal will be to build multilabel classification for the `code`
variable using `libelle` as feature.

## Enriching our test dataset

Unlike `Fasttext`, this package offers the possibility of having several
feature columns of different types (string for the text column and
additional variables in numeric form, for example). To illustrate that,
we propose the following enrichment of the example dataset:

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def categorize_surface(
    df: pd.DataFrame, surface_feature_name: int, like_sirene_3: bool = True
) -> pd.DataFrame:
    """
    Categorize the surface of the activity.

    Args:
        df (pd.DataFrame): DataFrame to categorize.
        surface_feature_name (str): Name of the surface feature.
        like_sirene_3 (bool): If True, categorize like Sirene 3.

    Returns:
        pd.DataFrame: DataFrame with a new column "surf_cat".
    """
    df_copy = df.copy()
    df_copy[surface_feature_name] = df_copy[surface_feature_name].replace("nan", np.nan)
    df_copy[surface_feature_name] = df_copy[surface_feature_name].astype(float)
    # Check surface feature exists
    if surface_feature_name not in df.columns:
        raise ValueError(f"Surface feature {surface_feature_name} not found in DataFrame.")
    # Check surface feature is a float variable
    if not (pd.api.types.is_float_dtype(df_copy[surface_feature_name])):
        raise ValueError(f"Surface feature {surface_feature_name} must be a float variable.")

    if like_sirene_3:
        # Categorize the surface
        df_copy["surf_cat"] = pd.cut(
            df_copy[surface_feature_name],
            bins=[0, 120, 400, 2500, np.inf],
            labels=["1", "2", "3", "4"],
        ).astype(str)
    else:
        # Log transform the surface
        df_copy["surf_log"] = np.log(df[surface_feature_name])

        # Categorize the surface
        df_copy["surf_cat"] = pd.cut(
            df_copy.surf_log,
            bins=[0, 3, 4, 5, 12],
            labels=["1", "2", "3", "4"],
        ).astype(str)

    df_copy[surface_feature_name] = df_copy["surf_cat"].replace("nan", "0")
    df_copy[surface_feature_name] = df_copy[surface_feature_name].astype(int)
    df_copy = df_copy.drop(columns=["surf_log", "surf_cat"], errors="ignore")
    return df_copy


def clean_and_tokenize_df(
    df,
    categorical_features=["EVT", "CJ", "NAT", "TYP", "CRT"],
    text_feature="libelle_processed",
    label_col="apet_finale",
):
    df.fillna("nan", inplace=True)

    df = df.rename(
        columns={
            "evenement_type": "EVT",
            "cj": "CJ",
            "activ_nat_et": "NAT",
            "liasse_type": "TYP",
            "activ_surf_et": "SRF",
            "activ_perm_et": "CRT",
        }
    )

    les = []
    for col in categorical_features:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        les.append(le)

    df = categorize_surface(df, "SRF", like_sirene_3=True)
    df = df[[text_feature, "EVT", "CJ", "NAT", "TYP", "SRF", "CRT", label_col]]

    return df, les


def stratified_split_rare_labels(X, y, test_size=0.2, min_train_samples=1):
    # Get unique labels and their frequencies
    unique_labels, label_counts = np.unique(y, return_counts=True)

    # Separate rare and common labels
    rare_labels = unique_labels[label_counts == 1]

    # Create initial mask for rare labels to go into training set
    rare_label_mask = np.isin(y, rare_labels)

    # Separate data into rare and common label datasets
    X_rare = X[rare_label_mask]
    y_rare = y[rare_label_mask]
    X_common = X[~rare_label_mask]
    y_common = y[~rare_label_mask]

    # Split common labels stratified
    X_common_train, X_common_test, y_common_train, y_common_test = train_test_split(
        X_common, y_common, test_size=test_size, stratify=y_common
    )

    # Combine rare labels with common labels split
    X_train = np.concatenate([X_rare, X_common_train])
    y_train = np.concatenate([y_rare, y_common_train])
    X_test = X_common_test
    y_test = y_common_test

    return X_train, X_test, y_train, y_test

def add_libelles(
    df: pd.DataFrame,
    df_naf: pd.DataFrame,
    y: str,
    text_feature: str,
    textual_features: list,
    categorical_features: list,
):
    missing_codes = set(df_naf["code"])
    fake_obs = df_naf[df_naf["code"].isin(missing_codes)]
    fake_obs[y] = fake_obs["code"]
    fake_obs[text_feature] = fake_obs[[text_feature]].apply(
        lambda row: " ".join(f"[{col}] {val}" for col, val in row.items() if val != ""), axis=1
    )
    df = pd.concat([df, fake_obs[[col for col in fake_obs.columns if col in df.columns]]])

    if textual_features is not None:
        for feature in textual_features:
            df[feature] = df[feature].fillna(value="")
    if categorical_features is not None:
        for feature in categorical_features:
            df[feature] = df[feature].fillna(value="NaN")

    print(f"\t*** {len(missing_codes)} codes have been added in the database...\n")
    return df

In [3]:
categorical_features = ["evenement_type", "cj",  "activ_nat_et", "liasse_type", "activ_surf_et", "activ_perm_et"]
text_feature = "libelle"
y = "apet_finale"
textual_features = None

naf2008 = pd.read_csv("https://minio.lab.sspcloud.fr/projet-ape/data/naf2008.csv", sep=";")
df = add_libelles(df, naf2008, y, text_feature, textual_features, categorical_features)

	*** 732 codes have been added in the database...



## Preprocessing

To reduce noise in text fields, we recommend pre-processing before
training a model with our package. We assume this preprocessing is
handled by the package user : this gives him the opportunity to control
data cleansing.

Here’s an example of the type of preprocessing that can be carried out
before moving on to the modeling phase

In [4]:
from torchTextClassifiers.utilities.preprocess import clean_text_feature
df["libelle_processed"] = clean_text_feature(df["libelle"])

/opt/python/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Right now, the model requires the label (variable y) to be a numerical
variable. If the label variable is a text variable, we recommend using
Scikit Learn’s
[LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)
to convert into a numeric variable. Using that function will give user
the possibility to get back labels from the encoder after running
predictions.

In [5]:
encoder = LabelEncoder()
df["apet_finale"] = encoder.fit_transform(df["apet_finale"])

The function `clean_and_tokenize_df` requires special `DataFrame`
formatting:

-   First column contains the processed text (str)
-   Next ones contain the “encoded” categorical (discrete) variables in
    int format

In [6]:
df, _ = clean_and_tokenize_df(df, text_feature="libelle_processed")
X = df[["libelle_processed", "EVT", "CJ", "NAT", "TYP", "CRT", "SRF"]].values
y = df["apet_finale"].values

/tmp/ipykernel_159490/2075507147.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("nan", inplace=True)


## Splitting in train-test sets

As usual in a learning approach, you need to break down your data into
learning and test/validation samples to obtain robust performance
statistics.
This work is the responsibility of the package’s users. Please make sure that np.max(y_train) == len(np.unique(y_train))-1 (i.e. your labels are well encoded, in a consecutive manner, starting from 0), and that all the possible labels appear at least once in the training set.

We provide the function stratified_train_test_split to match these requirements here..

In [7]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = stratified_split_rare_labels(X, y)

print(np.max(y_train) == len(np.unique(y_train))-1)

True


# Build the torchTextClassifiers FastText model (without training it)

There are several ways to define and train a FastText model in
this package.

We first show how to initialize the model and then afterwards build it.

`create_fasttext` function accepts the following parameters:

| Parameter | Meaning | Example Value |
|---------------------|------------------------------------------|----------|
| `num_tokens` | Number of rows in the embedding matrix (size of the vocabulary) | 100000 |
| `embedding_dim` | Dimension of the embedding (number of columns in the matrix) | 50 |
| `sparse` | Use sparse embedding for fast computation (PyTorch) | False |
| `categorical_embedding_dims` | Dimension of the embedding for categorical features | 10 |
| `min_count` | Minimum occurrences of a word in the corpus to be included | 1 |
| `min_n` | Minimum length of character n-grams | 3 |
| `max_n` | Maximum length of character n-grams | 6 |
| `len_word_ngrams` | Length of word n-grams | 3 |

In [8]:
from torchTextClassifiers import create_fasttext

parameters = {
    "num_tokens": 100000,
    "embedding_dim": 50,
    "sparse": False,
    "categorical_embedding_dims": 10,
    "min_count": 1,
    "min_n": 3,
    "max_n": 6,
    "len_word_ngrams": 3,
}

parameters_train = {
    "lr": 0.004,
    "num_epochs": 1,
    "batch_size": 256,
    "patience": 3   
}

model = create_fasttext(**parameters)

`model` is then a special `torchTextClassifiers` object:

In [10]:
type(model.classifier_wrapper)

torchTextClassifiers.classifiers.fasttext.wrapper.FastTextWrapper

As any `PyTorch` model, it accepts being save as a JSON for later on
use:

In [ ]:
model.to_json('torchTextClassifiers_config.json')
# Loading from JSON now works with the new API:
# from torchTextClassifiers import torchTextClassifiers
# loaded_model = torchTextClassifiers.from_json('torchTextClassifiers_config.json')

We can apply `build` to finally train our model. These are the
parameters accepted by the `build` method

| Parameter | Meaning | Example Value |
|---------------------|------------------------------------------|----------|
| `lr` | Learning rate | 0.004 |
| `num_epochs` | Number of training epochs | 1 |
| `batch_size` | Batch size for training | 256 |
| `patience` | Early stopping patience (number of epochs without improvement) | 3 |

We build the model using the training data. We have now access to the
tokenizer, the PyTorch model as well as a PyTorch Lightning module ready
to be trained. Note that Lightning is high-level framework for PyTorch
that simplifies the process of training, validating, and deploying
machine learning models.

In [11]:
model.build(X_train, y_train, lightning=True, lr=parameters_train.get("lr"))

2025-07-23 12:36:22 - torchTextClassifiers.classifiers.fasttext.model - num_rows is different from the number of tokens in the tokenizer. Using provided num_rows.


2025-07-23 12:36:22 - root - No scheduler parameters provided. Using defaults.


One can retrieve different objects from `model` instance:

-   `model.pytorch_model`
-   `model.tokenizer`
-   `model.lightning_module`

In [12]:
model.classifier_wrapper.pytorch_model

FastTextModel(
  (embeddings): EmbeddingBag(108297, 50, mode='mean', padding_idx=108296)
  (emb_0): Embedding(24, 10)
  (emb_1): Embedding(42, 10)
  (emb_2): Embedding(10, 10)
  (emb_3): Embedding(13, 10)
  (emb_4): Embedding(3, 10)
  (emb_5): Embedding(5, 10)
  (fc): Linear(in_features=60, out_features=732, bias=True)
)

In [13]:
model.classifier_wrapper.tokenizer

In [14]:
model.classifier_wrapper.lightning_module

FastTextModule(
  (model): FastTextModel(
    (embeddings): EmbeddingBag(108297, 50, mode='mean', padding_idx=108296)
    (emb_0): Embedding(24, 10)
    (emb_1): Embedding(42, 10)
    (emb_2): Embedding(10, 10)
    (emb_3): Embedding(13, 10)
    (emb_4): Embedding(3, 10)
    (emb_5): Embedding(5, 10)
    (fc): Linear(in_features=60, out_features=732, bias=True)
  )
  (loss): CrossEntropyLoss()
  (accuracy_fn): MulticlassAccuracy()
)

One can also retrieve more precise information regarding the tokenizer.
This can be useful to know how text is parsed before being given to the
neural network:

In [15]:
from pprint import pprint 
sentence = ["lorem ipsum dolor sit amet"]
pprint(model.classifier_wrapper.tokenizer.tokenize(sentence)[2][0])

{0: '</s>',
 369: 'sit',
 8402: 'lorem ipsum dolor',
 8477: '<sit',
 8958: 'sit amet </s>',
 9254: '<dol',
 15426: 'lorem>',
 17674: 'ame',
 19233: '<am',
 20208: 'dol',
 20956: 'amet>',
 21660: '<lor',
 22693: 'ips',
 24149: 'lore',
 27085: 'dolo',
 28043: '<si',
 29783: 'lorem ipsum',
 32285: 'met>',
 33686: 'or>',
 36146: 'ipsum dolor',
 37685: '<ipsum',
 37714: '<lorem',
 40715: '<lore',
 42140: 'dolor sit amet',
 43407: 'sit amet',
 43769: 'dolor>',
 44699: '<ips',
 45326: 'et>',
 46043: 'sit>',
 46176: 'ipsu',
 49083: 'psu',
 49236: 'orem>',
 50091: '<dolor',
 53987: 'um>',
 57650: 'it>',
 58295: 'olor>',
 60820: 'lor>',
 60827: 'ore',
 63114: 'sum>',
 65777: 'met',
 65864: '<amet',
 67731: 'em>',
 68083: 'olor',
 68290: 'orem',
 68834: 'psum',
 72463: 'rem',
 74123: 'ipsum>',
 74942: 'dolor sit',
 76898: 'lor',
 77899: '<dolo',
 82723: 'rem>',
 87932: '<ame',
 90231: 'amet </s>',
 93076: 'psum>',
 93114: '<ip',
 95253: 'ipsum dolor sit',
 97370: '<ipsu',
 99856: 'sum',
 102638: 

Saving parameters to JSON can also be done after building, but the model
needs to be rebuilt after loading.

In [ ]:
model.to_json('torchTextClassifiers_config.json')

In [ ]:
# Demonstrate the new JSON loading approach
from torchTextClassifiers import torchTextClassifiers

# Load model from JSON (works with new wrapper-based approach)
loaded_model = torchTextClassifiers.from_json('torchTextClassifiers_config.json')

print("✅ Model loaded from JSON successfully!")
print(f"Loaded wrapper type: {type(loaded_model.classifier_wrapper).__name__}")
print(f"Config parameters: embedding_dim={loaded_model.config.embedding_dim}, sparse={loaded_model.config.sparse}")

# The loaded model needs to be built before use
# loaded_model.build(X_train, y_train)

## New API Features

The updated `torchTextClassifiers` API provides more flexibility by allowing users to:

### 1. **Direct Wrapper Usage**
Create classifiers directly using wrapper classes, enabling custom implementations:

```python
from torchTextClassifiers import torchTextClassifiers
from torchTextClassifiers.classifiers.fasttext.wrapper import FastTextWrapper
from torchTextClassifiers.classifiers.fasttext.core import FastTextConfig

config = FastTextConfig(...)
wrapper = FastTextWrapper(config)
classifier = torchTextClassifiers(wrapper)
```

### 2. **Convenience Functions (Backward Compatible)**
The familiar convenience functions still work:

```python
from torchTextClassifiers import create_fasttext
classifier = create_fasttext(embedding_dim=50, sparse=False, ...)
```

### 3. **Enhanced JSON Support**
Improved serialization/deserialization that works with custom wrapper classes:

```python
# Save configuration
classifier.to_json('config.json')

# Load configuration (automatically detects wrapper type)
loaded_classifier = torchTextClassifiers.from_json('config.json')

# Or specify wrapper class explicitly
loaded_classifier = torchTextClassifiers.from_json('config.json', FastTextWrapper)
```

### 4. **Custom Classifier Support**
Users can now easily create their own classifier wrappers by inheriting from `BaseClassifierWrapper` and implementing the required methods.

## Alternative way to build torchTextClassifiers FastText

The training data is only useful to initialize the tokenizer, but
X_train and y_train are not needed to initialize the PyTorch model,
provided we give the right parameters to construct layer.

To highlight this, we provide a lower-level process to build the model
where one can first build the tokenizer, and then build the model with
custom architecture parameters.

**Note**: With the new API, you can also create classifiers directly using wrapper classes:

```python
from torchTextClassifiers import torchTextClassifiers
from torchTextClassifiers.classifiers.fasttext.wrapper import FastTextWrapper
from torchTextClassifiers.classifiers.fasttext.core import FastTextConfig

config = FastTextConfig(embedding_dim=50, sparse=False, ...)
wrapper = FastTextWrapper(config)
classifier = torchTextClassifiers(wrapper)
```

The tokenizer can be loaded **from the same JSON file** as the model
parameters, or initialized using the right arguments.

In [17]:
# Example of the new direct wrapper approach
from torchTextClassifiers import torchTextClassifiers
from torchTextClassifiers.classifiers.fasttext.wrapper import FastTextWrapper
from torchTextClassifiers.classifiers.fasttext.core import FastTextConfig

# Create configuration
config = FastTextConfig(
    embedding_dim=50,
    sparse=False,
    num_tokens=100000,
    min_count=1,
    min_n=3,
    max_n=6,
    len_word_ngrams=3,
    categorical_embedding_dims=10,
)

# Create wrapper and classifier
wrapper = FastTextWrapper(config)
direct_model = torchTextClassifiers(wrapper)

# Build the model
direct_model.build(X_train, y_train, lightning=True, lr=parameters_train.get("lr"))

print("✅ Direct wrapper model created successfully!")
print(f"Model type: {type(direct_model.classifier_wrapper).__name__}")
print(f"Config type: {type(direct_model.config).__name__}")

2025-07-23 12:38:33 - torchTextClassifiers.classifiers.fasttext.model - num_rows is different from the number of tokens in the tokenizer. Using provided num_rows.


2025-07-23 12:38:33 - root - No scheduler parameters provided. Using defaults.


✅ Direct wrapper model created successfully!
Model type: FastTextWrapper
Config type: FastTextConfig


In [18]:
del model

Let’s decompose our features in two group:

-   We have our textual feature stored in the first column of the
    features matrix
-   All other columns are categorical variables

In [18]:
training_text = X_train[:, 0].tolist()
categorical_variables = X_train[:, 1:]

We need to create a few variables that will be useful afterwards

In [19]:
CAT_VOCAB_SIZE = (np.max(categorical_variables, axis=0) + 1).astype(int).tolist()
NUM_CLASSES = len(np.unique(y_train))
NUM_CAT_VAR = categorical_variables.shape[1]

Now let’s come to the nitty gritty. There are several ways to create an
instance of the tokenizer.

First, we can create the tokenizer from :

-   model definition in the JSON file created beforehand
-   textual data in training dataset

In [ ]:
from torchTextClassifiers.classifiers.fasttext.tokenizer import NGramTokenizer
tokenizer = NGramTokenizer.from_json('torchTextClassifiers_config.json', training_text)

In [22]:
tokenizer.tokenize("Hello world")

([['<H>', '</s>', 'H </s>'],
  ['<e>', '</s>', 'e </s>'],
  ['<l>', '</s>', 'l </s>'],
  ['<l>', '</s>', 'l </s>'],
  ['<o>', '</s>', 'o </s>'],
  ['</s>'],
  ['<w>', '</s>', 'w </s>'],
  ['<o>', '</s>', 'o </s>'],
  ['<r>', '</s>', 'r </s>'],
  ['<l>', '</s>', 'l </s>'],
  ['<d>', '</s>', 'd </s>']],
 [tensor([40876,     0, 51965]),
  tensor([51907,     0, 77296]),
  tensor([74312,     0, 26137]),
  tensor([74312,     0, 26137]),
  tensor([ 9853,     0, 53786]),
  tensor([0]),
  tensor([29925,     0, 74978]),
  tensor([ 9853,     0, 53786]),
  tensor([ 8646,     0, 13223]),
  tensor([74312,     0, 26137]),
  tensor([ 89472,      0, 104945])],
 [{40876: '<H>', 0: '</s>', 51965: 'H </s>'},
  {51907: '<e>', 0: '</s>', 77296: 'e </s>'},
  {74312: '<l>', 0: '</s>', 26137: 'l </s>'},
  {74312: '<l>', 0: '</s>', 26137: 'l </s>'},
  {9853: '<o>', 0: '</s>', 53786: 'o </s>'},
  {0: '</s>'},
  {29925: '<w>', 0: '</s>', 74978: 'w </s>'},
  {9853: '<o>', 0: '</s>', 53786: 'o </s>'},
  {8646: '<r>

However, there is a more straightforward way to do: creating directly
the `NGramTokenizer` instance:

In [ ]:
tokenizer = NGramTokenizer(
    **parameters,
    training_text=training_text
    )

In [24]:
tokenizer.tokenize("Hello world")

([['<H>', '</s>', 'H </s>'],
  ['<e>', '</s>', 'e </s>'],
  ['<l>', '</s>', 'l </s>'],
  ['<l>', '</s>', 'l </s>'],
  ['<o>', '</s>', 'o </s>'],
  ['</s>'],
  ['<w>', '</s>', 'w </s>'],
  ['<o>', '</s>', 'o </s>'],
  ['<r>', '</s>', 'r </s>'],
  ['<l>', '</s>', 'l </s>'],
  ['<d>', '</s>', 'd </s>']],
 [tensor([40876,     0, 51965]),
  tensor([51907,     0, 77296]),
  tensor([74312,     0, 26137]),
  tensor([74312,     0, 26137]),
  tensor([ 9853,     0, 53786]),
  tensor([0]),
  tensor([29925,     0, 74978]),
  tensor([ 9853,     0, 53786]),
  tensor([ 8646,     0, 13223]),
  tensor([74312,     0, 26137]),
  tensor([ 89472,      0, 104945])],
 [{40876: '<H>', 0: '</s>', 51965: 'H </s>'},
  {51907: '<e>', 0: '</s>', 77296: 'e </s>'},
  {74312: '<l>', 0: '</s>', 26137: 'l </s>'},
  {74312: '<l>', 0: '</s>', 26137: 'l </s>'},
  {9853: '<o>', 0: '</s>', 53786: 'o </s>'},
  {0: '</s>'},
  {29925: '<w>', 0: '</s>', 74978: 'w </s>'},
  {9853: '<o>', 0: '</s>', 53786: 'o </s>'},
  {8646: '<r>

Why creating a `NGramTokenizer` separately ? Because model constructor
is now independent from training data:

In [ ]:
from torchTextClassifiers import build_fasttext_from_tokenizer

model = build_fasttext_from_tokenizer(
    tokenizer, 
    embedding_dim=parameters["embedding_dim"], 
    categorical_embedding_dims=parameters["categorical_embedding_dims"], 
    sparse=parameters["sparse"], 
    lr=parameters_train["lr"], 
    num_classes=NUM_CLASSES, 
    num_categorical_features=NUM_CAT_VAR, 
    categorical_vocabulary_sizes=CAT_VOCAB_SIZE
)

**Warning**:

If the PyTorch model building did not use the training data, please keep
in mind that its architecture (that you customize here) should match the
vocabulary size of the categorical variables and the total number of
class, otherwise the model will raise an error during training.

# Train a torchTextClassifiers FastText model directly

If no advanced customization or PyTorch tuning is necessary, there is a
direct way of training model.

In [20]:
model.train(
    X_train,
    y_train,
    X_test,
    y_test,
    num_epochs=parameters_train['num_epochs'],
    batch_size=parameters_train['batch_size'],
    patience_scheduler=parameters_train['patience'],
    patience_train=parameters_train['patience'],
    lr=parameters_train['lr'],
    verbose = True
)

2025-07-23 12:38:57 - torchTextClassifiers.torchTextClassifiers - Starting training process...
2025-07-23 12:38:57 - torchTextClassifiers.torchTextClassifiers - Running on: cpu
2025-07-23 12:38:57 - torchTextClassifiers.classifiers.fasttext.model - Creating DataLoader with 12 workers.
2025-07-23 12:38:57 - torchTextClassifiers.classifiers.fasttext.model - Creating DataLoader with 12 workers.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/python/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/python/lib/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 0: 100%|██████████| 34/34 [00:06<00:00,  4.89it/s, v_num=0, train_loss_step=5.480, val_loss=5.360, val_accuracy=0.345, train_loss_epoch=6.100, train_accuracy=0.178]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 34/34 [00:07<00:00,  4.71it/s, v_num=0, train_loss_step=5.480, val_loss=5.360, val_accuracy=0.345, train_loss_epoch=6.100, train_accuracy=0.178]

2025-07-23 12:39:07 - torchTextClassifiers.torchTextClassifiers - Training completed in 9.61 seconds.


# Load a trained model from a Lightning checkpoint

/! TOCOMPLETE

In [28]:
model.load_from_checkpoint(model.best_model_path) # or any other checkpoint path (string)

# Predicting from new labels

In [21]:
text = ["coiffeur, boulangerie, pâtisserie"] # one text description
X= np.array([[text[0], 0, 0, 0, 0, 0, 0]]) # our new entry
TOP_K = 5

pred, conf = model.predict(X, top_k=TOP_K)
pred_naf = encoder.inverse_transform(pred.reshape(-1))
subset = naf2008.set_index("code").loc[np.flip(pred_naf)]

for i in range(TOP_K-1, -1, -1):
    print(f"Prediction: {pred_naf[i]}, confidence:  {conf[0, i]}, description: {subset['libelle'][pred_naf[i]]}")

ValueError: not enough values to unpack (expected 2, got 1)

# Explainability

The `torchTextClassifiers` framework provides explainability features through the `predict_and_explain` method. This allows you to understand which parts of the input text contribute most to the model's predictions.

In [22]:
# Note: Visualization functions are not yet implemented in torchTextClassifiers
# The predict_and_explain method is available but visualization utilities need to be implemented

pred, conf, all_scores, all_scores_letters = model.predict_and_explain(X)

# TODO: Implement visualization functions
# visualize_word_scores(all_scores, text, pred_naf.reshape(1, -1))
# visualize_letter_scores(all_scores_letters, text, pred_naf.reshape(1, -1))

print("Predictions with explanations:")
print(f"Predicted classes: {pred}")
print(f"Confidence scores: {conf}")
print(f"Word-level explanation scores shape: {all_scores.shape if hasattr(all_scores, 'shape') else type(all_scores)}")
print(f"Letter-level explanation scores shape: {all_scores_letters.shape if hasattr(all_scores_letters, 'shape') else type(all_scores_letters)}")

Predictions with explanations:
Predicted classes: tensor([[727]])
Confidence scores: tensor([[0.5400]])
Word-level explanation scores shape: <class 'list'>
Letter-level explanation scores shape: torch.Size([1, 1, 31])
